### Training a Custom Deep Learning Model

---

## Imports and Dependencies

In [1]:
import os
import sys
import json
import numpy as np
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import SGD

/Users/gathanasiou/oxcitas/projects/ML/MRI/deepMRI/deepMRI_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Add the parent directory of `src` to the Python path
sys.path.append(os.path.abspath('../'))

In [13]:
# Custom imports (placeholders for demonstration)
import src.constants as C
from src.data import Dataset
from src.models import Custom3DCNN, SFCN, resnet18
from src.utils import AWDLoss, KLDivLoss, LocalMAE, augment, scheduler, discretize_ages, save_subject_ids, load_scan

## Configuration
Setting up constants and a small demonstration configuration.

In [14]:
# Constants for demonstration
CSIZE = 128  # Reduced size for faster processing
BATCH_SIZE = 4
EPOCHS = 2
LEARNING_RATE = 0.001
BINS = 10
USE_SOFT_LABELS = False

In [15]:
# Load training configuration
config_path = Path('../configs/training_config.json').expanduser().resolve()
with open(config_path, 'r') as config_file:
    config = json.load(config_file)

In [16]:
# Extract configuration parameters
PROJECT_NAME = config['project_name']
OUTPUT_DIRECTORY = Path(config['output_directory']).expanduser().resolve()
USE_SOFT_LABELS = config['use_soft_labels']
print(PROJECT_NAME) 
print(OUTPUT_DIRECTORY) 
print(USE_SOFT_LABELS)

deepMRI@demo
/Users/gathanasiou/oxcitas/UKB-gen-clocks/DL-apporach/configs/dataset_configs
False


---

## Back up functions

In [17]:
def prepare_data(dataset):
    """
    Prepare the dataset by appending file extensions to scan IDs.
    """
    dataset.scanIDs = [scan + '.mgz' for scan in dataset.scanIDs]
    return dataset

In [18]:
def create_data_batches(X_train, y_train, X_test, y_test, batch_size, soft_labels=False):
    """
    Create TensorFlow data batches for training and testing.
    """
    if soft_labels:
        train_labels = np.array([item for item in y_train])
        test_labels = np.array([item for item in y_test])
    else:
        train_labels = np.array(y_train)
        test_labels = np.array(y_test)

    train_scans = tf.data.Dataset.from_tensor_slices((X_train, train_labels))
    test_scans = tf.data.Dataset.from_tensor_slices((X_test, test_labels))

    train_scans = train_scans.map(load_scan, num_parallel_calls=tf.data.AUTOTUNE).cache()
    test_scans = test_scans.map(load_scan, num_parallel_calls=tf.data.AUTOTUNE)
    
    BUFFER_SIZE = 500
    train_batches = (
        train_scans
        .cache()
        .shuffle(BUFFER_SIZE)
        .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .filter(lambda x, y: tf.shape(x)[0] > 1)  # Filter out small batches
        .repeat()
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )

    test_batches = test_scans.batch(batch_size).filter(lambda x, y: tf.shape(x)[0] > 1)  # Filter out small batches

    return train_batches, test_batches

In [19]:
def compile_model_real_labels(model, learning_rate):
    """
    Compile the model for real labels with the custom AWD loss function and Adam optimizer.
    """
    awdLoss = AWDLoss(num_bins=C.BINS, initial_alpha=C.ALPHA, initial_beta=C.BETA)
    
    def awd_loss(y_true, y_pred):
        return awdLoss(y_true, y_pred)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=awd_loss,
                  metrics=['mean_absolute_error', 'mean_squared_error', LocalMAE(num_bins=C.BINS)])
    return model

## Data Preparation
Loading real data based on the provided configuration.

In [20]:
# Load dataset configuration
DATASET_CONFIG_PATH = OUTPUT_DIRECTORY / ('dataset_config_soft.json' if USE_SOFT_LABELS else 'dataset_config_real.json')
data = Dataset.from_file(DATASET_CONFIG_PATH)
data = prepare_data(data)

10
Target path:  ../configs/targets/real_targets.json


In [21]:
C.TEST_SIZE

0.2

In [22]:
# Prepare train-test splits
for fold_num, (X_train, X_test, y_train, y_test) in enumerate(data.train_test_subjID_iterator(), 1):
    print(f"Training fold {fold_num}...")
    print(f"Number of cases in X_train: {len(X_train)}")
    print(f"Number of cases in X_test: {len(X_test)}")

    train_batches, test_batches = create_data_batches(X_train, y_train, X_test, y_test, BATCH_SIZE, soft_labels=USE_SOFT_LABELS)
    break  # Only load the first fold for this demonstration

Total subjects: 9
Training subjects: 8, Test subjects: 1
Train Scans: 9, Train Labels: 9
Test Scans: 1, Test Labels: 1
Formatted Train Labels Shape: (9,), Test Labels Shape: (1,)
Training fold 1...
Number of cases in X_train: 9
Number of cases in X_test: 1


In [38]:
for fold_num, (X_train, X_test, y_train, y_test) in enumerate(data.train_test_subjID_iterator(), 1):
    print(f"Training fold {fold_num}...")
    print(f"Number of cases in X_train: {len(X_train)}")
    print(f"Number of cases in X_test: {len(X_test)}")

    # y_train_bins, train_bins = discretize_ages(y_train)
    # y_test_bins, test_bins = discretize_ages(y_test)

    model_pick = C.RESNET
    if USE_SOFT_LABELS:
        print("Soft labeling case initialized...")
        if model_pick:
            print("Using ResNet model...")
            model = resnet18(input_shape=(C.CSIZE, C.CSIZE, C.CSIZE, 1),
                     num_classes=int((C.HGH_BIN - C.LOW_BIN) / C.STEP_BIN), 
                     channel_size=[64, 64, 128, 256, 512], 
                     dropout=True)
        else:
            print("Using SFCN model...")
            model = SFCN(input_shape=(C.CSIZE, C.CSIZE, C.CSIZE, 1), 
                        channels=[32, 64, 128, 256, 256, 64], 
                        output_dim=int((C.HGH_BIN - C.LOW_BIN) / C.STEP_BIN), 
                        use_dropout=True, 
                        csize=C.CSIZE).build_model()
        model = compile_model_soft_labels(model, C.LEARNING_RATE)
        train_batches, test_batches = create_data_batches(X_train, y_train, X_test, y_test, C.BATCH_SIZE, soft_labels=True)
    else:
        print("Regression case initialized...")
        model = Custom3DCNN(input_shape=(C.CSIZE, C.CSIZE, C.CSIZE, 1),
                            depth=C.DEPTH, 
                            initial_filters=C.INITIAL_FILTERS, 
                            l2_strength=C.L2_STRENGTH).build_model()
        model = compile_model_real_labels(model, C.LEARNING_RATE)
        train_batches, test_batches = create_data_batches(X_train, y_train, X_test, y_test, C.BATCH_SIZE)

    STEPS_PER_EPOCH = len(X_train) // C.BATCH_SIZE
    checkpoint_dir_fold = os.path.join('../data/', f"model_output/fold_{fold_num}")
    os.makedirs(checkpoint_dir_fold, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir_fold, f"best_weights_fold_{fold_num}.weights.h5")

    if C.LOAD_WEIGHTS and os.path.exists(checkpoint_path):
        print(f"Loading weights from {checkpoint_path}...")
        model.load_weights(checkpoint_path)

    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only=True,
                                 monitor='val_loss',
                                 mode='min',
                                 save_best_only=True,
                                 verbose=1)
    
    lr_scheduler = LearningRateScheduler(scheduler)

    log_file_path = os.path.join('../data/', 'logs', f'metrics_log_fold_{fold_num}.txt')
    os.makedirs(os.path.dirname(log_file_path), exist_ok=True)
    
    save_subject_ids(checkpoint_dir_fold, fold_num, X_train, X_test)

    callbacks = [checkpoint, lr_scheduler]

    history = model.fit(train_batches,
                        epochs=C.EPOCHS,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        validation_data=test_batches,
                        verbose=1,
                        callbacks=callbacks)

Total subjects: 9
Training subjects: 8, Test subjects: 1
Train Scans: 9, Train Labels: 9
Test Scans: 1, Test Labels: 1
Formatted Train Labels Shape: (9,), Test Labels Shape: (1,)
Training fold 1...
Number of cases in X_train: 9
Number of cases in X_test: 1
Regression case initialized...
Epoch 1/10


TypeError: object of type 'ResourceVariable' has no len()

---

## Model Compilation
Compiling a model for demonstration purposes.

In [23]:
# Model selection and compilation
if USE_SOFT_LABELS:
    print("Using soft labels...")
    model = resnet18(input_shape=(CSIZE, CSIZE, CSIZE, 1),
                     num_classes=BINS,
                     channel_size=[64, 64, 128, 256, 512],
                     dropout=True)
    model = compile_model_soft_labels(model, LEARNING_RATE)
else:
    print("Using real labels...")
    model = Custom3DCNN(input_shape=(CSIZE, CSIZE, CSIZE, 1), depth=3, initial_filters=32, l2_strength=0.01).build_model()
    model = compile_model_real_labels(model, LEARNING_RATE)

Using real labels...


---

## Training the Model
Training the model on a small dataset for a few epochs.

In [24]:
# Callbacks for training
checkpoint_path = OUTPUT_DIRECTORY / "best_model.keras"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor="val_loss", mode="min")
lr_scheduler = LearningRateScheduler(scheduler)

In [26]:
# Calculate steps per epoch
STEPS_PER_EPOCH = len(X_train) // BATCH_SIZE

In [27]:
# Train the model
history = model.fit(train_batches,
                    validation_data=test_batches,
                    epochs=EPOCHS,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    callbacks=[checkpoint, lr_scheduler])

Epoch 1/2


TypeError: object of type 'ResourceVariable' has no len()

---

## Evaluation and Predictions
Evaluate the trained model and demonstrate predictions.

In [21]:
# Evaluate the model
loss, mae, mse = model.evaluate(test_dataset)
print(f"Evaluation Results - Loss: {loss}, MAE: {mae}, MSE: {mse}")

TypeError: object of type 'ResourceVariable' has no len()

In [22]:
# Predictions
predictions = model.predict(test_dataset)
print(f"Sample Predictions: {predictions[:5]}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 263ms/step
Sample Predictions: [[-0.04665842]
 [-0.04773519]
 [-0.04873811]
 [-0.04604891]
 [-0.04694086]]


---

## Visualization
Visualize augmentation and training results.

In [23]:
import matplotlib.pyplot as plt

In [24]:
# Plot augmentation example
plt.figure()
plt.imshow(augmented_sample[:, :, CSIZE // 2, 0], cmap="gray")  # Visualize the middle slice
plt.title("Augmented Sample")
plt.show()

NameError: name 'augmented_sample' is not defined

<Figure size 640x480 with 0 Axes>

In [25]:
# Plot training history
plt.figure()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Training History")
plt.legend()
plt.show()

NameError: name 'history' is not defined

<Figure size 640x480 with 0 Axes>

---

## Conclusion
This notebook demonstrates a simplified training pipeline using synthetic data. Adapt this template with real data and models for your specific use case.